In [1]:
import json
import torch
from tqdm import tqdm
import re
from unsloth import FastLanguageModel
from collections import defaultdict

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-10-15 23:02:14.590797: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-15 23:02:18.718736: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


INFO 10-15 23:02:46 [__init__.py:216] Automatically detected platform cuda.
ERROR 10-15 23:02:49 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
Switching to PyTorch attention since your Xformers is broken.

Requires Flash-Attention version >=2.7.1,<=2.8.2 but got 2.8.3.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
input_filename = '/home/jupyter/datasphere/project/evaluation_results_fair.json'
with open(input_filename, 'r') as f:
    json_data = json.load(f)

sentence_pairs = []
for result in json_data["results"]:
  sentence_pairs.append([result["ground_truth"] if type(result["ground_truth"]).__name__ == "str" else result["ground_truth"][0], result["prediction"] ])


In [3]:
def make_prompt_en(gt, pred):
    return f"""
Reference Answer: "{gt}"
Generated Answer: "{pred}"

Analyze these two texts and provide your rating with a justification.
Your justification:"""

In [4]:
system_prompt_en = """You are a meticulous and strict AI evaluator. Your goal is to score the "Generated Answer" based on its factual accuracy and completeness when compared to the "Reference Answer".

Follow these rules precisely:
1.  **NON-NEGOTIABLE RULE — Factual Accuracy is Paramount:** This rule OVERRIDES all others. Pay extremely close attention to numbers, dates, names, statistics, and other specific facts. A single factual error that contradicts the Reference Answer makes the entire answer wrong and **must result in a score of 0.0**, regardless of how well-structured or detailed the answer is.
2.  **Completeness:** Check if the Generated Answer includes all key information present in the Reference Answer. Missing a crucial detail should lower the score.
3.  **Correct Refusal:** If the Generated Answer correctly states that information is not available in the source document, this is a GOOD answer. It shows honesty and avoids hallucination.
4.  **Language Invariance:** Evaluate based on meaning, regardless of the language (English, Russian, etc.). The rules are the same for all languages.
5.  **Handling Minimalist Reference Answers:** If the Reference Answer is very short (e.g., "Yes", "No", "Да", "Нет", or a single number), it represents the core truth, not the full context. A Generated Answer that correctly states this core truth and adds relevant, descriptive details should be considered high-quality. Do not penalize it for providing extra information unless that information contradicts the core truth.

First, write a brief justification for your assessment in 1-2 sentences. After the justification, on a new line, write "Rating:" followed by your score on a scale from 0 to 5, with a step of 0.25.

**CRITICAL OUTPUT FORMATTING RULE:** Regardless of the language of the texts being evaluated (English, Russian, etc.), your justification must be in English, and the final line with the score **MUST ALWAYS** use the exact format `Rating: [score]`.

**Scoring Rubric:**
- **5.0:** Perfect match. All facts are correct and all key information from the reference is included.
- **4.0 - 4.75:** Factually correct, but might miss a very minor detail or include extra, harmless information.
- **2.0 - 3.75:** Partially correct. Captures the main idea but has minor factual errors or is missing significant information.
- **0.0 - 1.75:** Mostly incorrect. Contains major factual errors that contradict the reference, or is severely incomplete. A score of 0.0 is for answers that are completely wrong or opposite in meaning.

---
**Examples of Application:**

**Example of a Factual Error (Score 0.0):**
Reference Answer: "The delivery will be on May 10th."
Generated Answer: "Your package will arrive on May 11th."
Justification: The Generated Answer contains a direct factual error regarding the date, which contradicts the Reference Answer.
Rating: 0.0

**Example of an Incomplete Answer (Score 3.0):**
Reference Answer: "The team needs a designer, a programmer, and a manager."
Generated Answer: "The team is looking for a designer and a programmer."
Justification: The Generated Answer correctly identifies two roles but omits the manager, making it significantly incomplete.
Rating: 3.0

**Example of a Correct Refusal (Score 5.0):**
Reference Answer: "The event is on Tuesday."
Generated Answer: "The provided document does not state the day of the event."
Justification: The model correctly identified that the information was not in the source. This is a perfect score for honesty.
Rating: 5.0

**Example of Handling a Minimalist Answer (Score 5.0):**
Reference Answer: "Да"
Generated Answer: "Да, на фотографии виден один человек на футбольном поле."
Justification: The Generated Answer correctly confirms the core fact ("Да") from the Reference Answer and adds relevant, descriptive context. This is a high-quality, helpful answer.
Rating: 5.0
"""

In [5]:
import gc
model = None
gc.collect()
torch.cuda.empty_cache()

In [6]:
MODEL_NAME = "unsloth/Qwen3-30B-A3B-bnb-4bit"

In [ ]:

print(f"Загрузка модели: {MODEL_NAME}")
max_seq_length = 2500
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = max_seq_length,
    dtype = None,
    device_map="balanced",
    load_in_4bit = True,
)


Загрузка модели: unsloth/Qwen3-30B-A3B-bnb-4bit
==((====))==  Unsloth 2025.10.1: Fast Qwen3_Moe patching. Transformers: 4.56.2. vLLM: 0.11.0.
   \\   /|    Tesla V100-PCIE-32GB. Num GPUs = 1. Max memory: 31.739 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 13/13 [16:40<00:00, 76.95s/it]
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 4318bf13-247f-4524-93dc-a6c31975b751)')' thrown while requesting HEAD https://huggingface.co/unsloth/qwen3-30b-a3b/resolve/main/generation_config.json
Retrying in 1s [Retry 1/5].


In [ ]:
class ScoreLogitsProcessor(torch.nn.Module):

    def __init__(self, tokenizer, marker="\nRating:"):
        super().__init__()
        self.tokenizer = tokenizer
        
        self.marker_tokens = self.tokenizer.encode(marker, add_special_tokens=False)
        self.marker_len = len(self.marker_tokens)
        
        allowed_chars = "0123456789."
        self.allowed_token_ids = []
        for char in allowed_chars:
            token_id = self.tokenizer.encode(char, add_special_tokens=False)
            if token_id:
                self.allowed_token_ids.append(token_id[0])
        
        self.allowed_token_ids.append(self.tokenizer.eos_token_id)
        self.allowed_token_ids = list(set(self.allowed_token_ids))

        self.activated = False

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
        
        if not getattr(self, 'activated_batch', False):
            self.activated_batch = [False] * scores.shape[0]

        for i in range(input_ids.shape[0]):
            if self.activated_batch[i]:
                mask = torch.full_like(scores[i], -float("inf"))
                mask[self.allowed_token_ids] = 0
                scores[i] = scores[i] + mask
                continue

            generated_tokens = input_ids[i]
            if len(generated_tokens) >= self.marker_len:
                last_tokens = generated_tokens[-self.marker_len:].tolist()
                if last_tokens == self.marker_tokens:
                    self.activated_batch[i] = True
                    mask = torch.full_like(scores[i], -float("inf"))
                    mask[self.allowed_token_ids] = 0
                    scores[i] = scores[i] + mask # Применяем маску
                    
        return scores

    def reset(self):
        self.activated_batch = False

In [ ]:

gc.collect()
torch.cuda.empty_cache()

In [ ]:
import re
BATCH_SIZE=40
score_processor = ScoreLogitsProcessor(tokenizer, marker="\nRating:")

llm_responses_en = []
with torch.no_grad():
    for i in tqdm(range(0, len(sentence_pairs), BATCH_SIZE)):
        score_processor.reset()
        
        batch_pairs = sentence_pairs[i:i + BATCH_SIZE]
        batch_messages = []
        for gt, pred in batch_pairs:
            messages = [{"role": "system", "content": system_prompt_en}, {"role": "user", "content": make_prompt_en(gt, pred)}]
            batch_messages.append(messages)

        templated_texts = [tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True, enable_thinking=False) for msgs in batch_messages]
        model_inputs = tokenizer(templated_texts, return_tensors="pt", padding=True).to("cuda")
        
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=200,
            do_sample=False,
            num_beams=1,
            logits_processor=[score_processor],
            eos_token_id=tokenizer.eos_token_id,
        )
        
        input_ids_len = model_inputs.input_ids.shape[1]
        decoded_responses = tokenizer.batch_decode(generated_ids[:, input_ids_len:], skip_special_tokens=True)
        llm_responses_en.extend(decoded_responses)

clean_responses = []
final_scores = []

for resp in llm_responses_en:
    clean_part = resp.split('</think>')[-1].strip() if '</think>' in resp else resp.strip()
    clean_responses.append(clean_part)
    
    match = re.search(r'Rating:\s*([\d\.]+)', clean_part)
    if match:
        final_scores.append(float(match.group(1)))
    else:
        final_scores.append(None)

print("\nExample final scores:")
print(final_scores[:10])

  0%|          | 0/3 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true 


Example final scores:
[0.0, 5.0, 0.0, 2.0, 5.0, 0.0, 5.0, 5.0, 5.0, 5.0]


In [ ]:
output_filename = "evaluation_results_fair_with_llm_analysis.json"

print("Постобработка: извлечение оценок и обоснований...")
final_scores = []
final_justifications = []

for resp in llm_responses_en:
    parts = resp.split('\nRating:')
    if len(parts) == 2:
        justification = parts[0].strip()
        score_str = parts[1].strip()
        try:
            score = float(score_str)
            final_scores.append(score)
            final_justifications.append(justification)
        except (ValueError, IndexError):
            final_scores.append(None)
            final_justifications.append(f"Error: Could not parse score from '{score_str}'. Full response: {resp}")
    else:
        final_scores.append(None)
        final_justifications.append(f"Error: Model output format incorrect. Full response: {resp}")

print("Обновление JSON-файла новыми данными...")
with open(input_filename, 'r', encoding='utf-8') as f:
    data_to_save = json.load(f)

if not (len(data_to_save['results']) == len(final_scores) == len(final_justifications)):
    print("КРИТИЧЕСКАЯ ОШИБКА: Размеры данных не совпадают! Файл не будет сохранен.")
else:
    for i, result in enumerate(data_to_save['results']):
        result['llm_score'] = final_scores[i]
        result['llm_justification'] = final_justifications[i]

    valid_scores = [s for s in final_scores if s is not None]
    if valid_scores:
        data_to_save['overall_metrics']['llm_average_score'] = sum(valid_scores) / len(valid_scores)
        data_to_save['overall_metrics']['llm_accuracy_05'] = sum(1 for s in valid_scores if s >= 0.5) / len(valid_scores)
    else:
        data_to_save['overall_metrics']['llm_average_score'] = 0
        data_to_save['overall_metrics']['llm_accuracy_05'] = 0

    scores_by_source = defaultdict(list)
    for result in data_to_save['results']:
        score = result.get('llm_score')
        if score is not None:
            scores_by_source[result['source']].append(score)
    
    for source, scores in scores_by_source.items():
        if source in data_to_save['metrics_by_source']:
            data_to_save['metrics_by_source'][source]['llm_average_score'] = sum(scores) / len(scores)
            data_to_save['metrics_by_source'][source]['llm_accuracy_05'] = sum(1 for s in scores if s >= 0.5) / len(scores)

    print(f"Сохранение обогащенных результатов в файл: {output_filename}")
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(data_to_save, f, indent=2, ensure_ascii=False)
    print("Готово! Новый файл с метриками side-by-side успешно создан.")

Постобработка: извлечение оценок и обоснований...
Обновление JSON-файла новыми данными...
Сохранение обогащенных результатов в файл: evaluation_results_fair_with_llm_analysis.json
Готово! Новый файл с метриками side-by-side успешно создан.


In [16]:
print(sum(final_scores)/ (5 *len(final_scores)))

0.6405


In [ ]:
llm_responses_en[10:20]

['The Generated Answer provides a detailed analysis and concludes that there is "one" rubber object of the same size as the red cylinder, while the Reference Answer is simply "3". The Generated Answer\'s conclusion directly contradicts the Reference Answer, making it factually incorrect. The error is major and fundamental, as it provides a completely different numerical answer than the reference.\n\nRating: 0.0',
 'The Generated Answer correctly identifies that the small green object has the shape of a "сфера" (sphere), which matches the Reference Answer. The answer provides a detailed description of the image, including the positioning and other objects, but the key fact about the shape of the small green object is accurate. There are no factual errors, and the core information from the Reference Answer is included.\n\nRating: 5.0',
 'The Generated Answer correctly identifies that Language Level 3 offers English and Spanish lessons, aligning with the Reference Answer. However, it adds